## CLIO Project
Candidate: Geoffrey Shmigelsky

Tuesday, Oct 29, 2019



In [1]:
import pandas as pd

In [13]:
# set to False to process Test datafile instead
TRAIN = True 
# TRAIN = False

# training dataset is inbalanced 7:1 (NO:YES) for Conversion classification
REPLICATE = 0 # number of times to duplicate training Yes data, max is 7

# Output:
# Will create one of two files, with engineered features for time 
#   - Invalid entries removed
#   - Pre conversions removed, only in Training set
# File #1: train_clean.tab (TRAIN = True)
# File #2: test_clean.tab (TRAIN = False)


### Read in the CSV file
Train has a Y label, Test does not.
The column idx will serve as an index, but carries no meaning.

In [19]:
if TRAIN:
    df = pd.read_csv("train.csv", index_col='idx')
else:
    df = pd.read_csv("test.csv", index_col='idx')
    
#TODO: Process missing values here, let Orange for it for now   
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 745 to 270
Data columns (total 10 columns):
time_to_first_matter           1000 non-null float64
time_to_first_time_entry       1000 non-null float64
time_to_first_bill             1000 non-null float64
time_to_second_user            1000 non-null float64
page_views_in_first_hour       1000 non-null float64
page_views_in_first_day        1000 non-null float64
page_views_in_first_7_days     1000 non-null float64
page_views_in_first_14_days    1000 non-null float64
time_to_conversion             1000 non-null float64
conversion_value               1000 non-null float64
dtypes: float64(10)
memory usage: 85.9 KB


### Remove Invalid Cases

Remove rows where there are no page views, these would indicate bad data.

In [20]:
print(f"Rows including bad data: {len(df)}")
df_bad = df.isna()['page_views_in_first_hour']
print(f"Rows with bad data     : {df_bad.sum()}")
df = df[~df_bad]
print(f"Cleaned rows remaining : {len(df)}")

Rows including bad data: 1000
Rows with bad data     : 0
Cleaned rows remaining : 1000


### Conversion Cleanup and Engineering 

If a prospect's time_to_conversion occurs before a the first matter, then the prospect has already converted 

The time_to_first_matter and time_to_conversion are reducable to a single True/False representing a successful conversion. 



In [16]:
if TRAIN:
    print(f"Total number of rows          : {len(df)}")

    df['Conversion'] = df['time_to_conversion'] > 0
    # df['Conversion'] = df['Conversion'].astype(int)
    df['PreConversion'] = df['time_to_first_matter'] > df['time_to_conversion']
    print(f"Total Conversions             : {df['Conversion'].sum()}")
    print(f"Number of PreConversions      : {df['PreConversion'].sum()}")
    print(f"Number of True Conversions    : {df['Conversion'].sum()- df['PreConversion'].sum()}")

    df = df[~df['PreConversion']]

    print(f"Rows left minus PreConversions: {len(df)}")      

Total number of rows          : 1000
Total Conversions             : 171
Number of PreConversions      : 351
Number of True Conversions    : -180
Rows left minus PreConversions: 649


### Time Normalization
First, average page views by the hour.
Second, compute the average change by time frame.

In [6]:
df['page_views_in_first_day_norm'] = df.page_views_in_first_day / 7.0
df['page_views_in_first_7_days_norm'] = df.page_views_in_first_7_days / (7.0*24.0)
df['page_views_in_first_14_days_norm'] = df.page_views_in_first_7_days / (14.0*24.0)

df['time_delta_day'] = df.page_views_in_first_hour - df.page_views_in_first_day_norm
df['time_delta_week'] = df.page_views_in_first_day_norm - df.page_views_in_first_7_days_norm
df['time_delta_2weeks'] = df.page_views_in_first_7_days_norm - df.page_views_in_first_14_days_norm


### Match up Yes and No Counts

There is an imbalance of No Conversions to Yes Conversions of 7 to 1.

A quick workaround is to duplicate the Yes data seven times. This will allow for a more intuitive understanding of the Confusion Matrix.

Finalize Y label as "Yes" and "No" for ease of reading in Orange.

In [7]:
if TRAIN:
    df['Conversion'] = df['Conversion'].replace({True: 'Yes', False: 'No'})

    yes = df[df['Conversion'] == 'Yes']
    no = df[df['Conversion'] == 'No']
    # replicate YES data to match claissification counts, if wanted
    for _ in range(REPLICATE):
        df = df.append(yes)

    print(len(df))
    print(f"No Length = {len( df[df['Conversion'] == 'No'] )}")
    print(f"Yes Length = {len(df[df['Conversion'] == 'Yes'] )}")
          
    df['Conversion'] = df.pop('Conversion')  # make last columns

914
No Length = 801
Yes Length = 113


In [8]:
if TRAIN:
    yes.describe().T

In [9]:
if TRAIN: 
    no.describe().T

In [10]:
print(df.columns)
df.describe().T


Index(['time_to_first_matter', 'time_to_first_time_entry',
       'time_to_first_bill', 'time_to_second_user', 'page_views_in_first_hour',
       'page_views_in_first_day', 'page_views_in_first_7_days',
       'page_views_in_first_14_days', 'time_to_conversion', 'conversion_value',
       'PreConversion', 'page_views_in_first_day_norm',
       'page_views_in_first_7_days_norm', 'page_views_in_first_14_days_norm',
       'time_delta_day', 'time_delta_week', 'time_delta_2weeks', 'Conversion'],
      dtype='object')


,count,mean,std,min,25%,50%,75%,max
time_to_first_matter,379.0,81816.358839,198153.789905,77.000000,348.500000,1126.000000,29915.000000,1.130506e+06
time_to_first_time_entry,313.0,87472.575080,195930.815550,15.000000,480.000000,1900.000000,66567.000000,1.207301e+06
time_to_first_bill,164.0,164917.195122,268088.212590,217.000000,1193.500000,6383.000000,235260.250000,1.204219e+06
time_to_second_user,84.0,226122.214286,330303.572491,101.000000,815.250000,21457.500000,349205.250000,1.198713e+06
page_views_in_first_hour,914.0,19.936543,23.257648,1.000000,5.000000,12.000000,25.000000,1.490000e+02
page_views_in_first_day,914.0,36.717724,67.840603,1.000000,7.000000,15.000000,36.000000,6.730000e+02
page_views_in_first_7_days,914.0,66.492341,147.253008,1.000000,8.000000,19.000000,55.750000,1.813000e+03
page_views_in_first_14_days,914.0,85.698031,196.621187,1.000000,8.000000,20.000000,66.000000,2.117000e+03
time_to_conversion,113.0,523400.911504,334743.882297,528.000000,254333.000000,544740.000000,683947.000000,1.204386e+06
conversion_value,113.0,101.687965,108.349874,29.590000,49.000000,62.100000,117.300000,8.280000e+02


### Export to CSV

Drop unused columns and PreConversion rows.


In [11]:
if TRAIN: 
    df.drop(['time_to_conversion', 'conversion_value', 'PreConversion'], axis=1, inplace=True)
    df.to_csv("train_clean.tab", sep='\t', index=False)
else:
    df.to_csv("test_clean.tab", sep='\t', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 914 entries, 745 to 270
Data columns (total 15 columns):
time_to_first_matter                379 non-null float64
time_to_first_time_entry            313 non-null float64
time_to_first_bill                  164 non-null float64
time_to_second_user                 84 non-null float64
page_views_in_first_hour            914 non-null float64
page_views_in_first_day             914 non-null float64
page_views_in_first_7_days          914 non-null float64
page_views_in_first_14_days         914 non-null float64
page_views_in_first_day_norm        914 non-null float64
page_views_in_first_7_days_norm     914 non-null float64
page_views_in_first_14_days_norm    914 non-null float64
time_delta_day                      914 non-null float64
time_delta_week                     914 non-null float64
time_delta_2weeks                   914 non-null float64
Conversion                          914 non-null object
dtypes: float64(14), object(1)
memory us

### TODO:
1. QC on matter is superset to time entry, time entry is superset to first bill
2. Confirm both converstion inputs have values, ratio is 1:1